# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1o2lpxoi9heyQV1hIlsHXWSfDkBPtze-V?usp=sharing

Para isso, vamos criar uma automação web:

+ Usaremos o selenium: `conda install -c conda-forge selenium`
+ Importante: baixar o webdriver e colocar ele na pasta onde o anaconda3 está instalado
    + Google Chrome: chromedriver
    + Firefox: geckodriver

Passo a passo

+ Passo 1: pegar cotação do Dolar

+ Passo 2: pegar cotação do Euro

+ Passo 3: pegar cotação do Ouro

+ Passo 4: importar a tabela de produtos

+ Passo 5: Recalcular o preço de cada produto

+ Passo 6: Salvar os novos preços dos produtos

In [25]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# headless mode || segundo plano
# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.headless = True # also works
# nav = webdriver.Chrome(options=chrome_options)

navegador = webdriver.Chrome()

def find_price(input_text):
    navegador.get('https://www.google.com')
    navegador.find_element_by_xpath(
        '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(input_text) # reach the input search and write something
    navegador.find_element_by_xpath(
        '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER) # reach the input search and press ENTER
    
    return float(navegador.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value'))
    
# Passo 1

dolar_price = find_price('cotação dolar')
# print('dolar: ' + str(dolar_price) + ' ' + str(type(dolar_price)))

# Passo 2

euro_price = find_price('cotacao euro')
# print('euro: ' + str(euro_price) + ' ' + str(type(euro_price)))

# Passo 3

navegador.get('https://www.melhorcambio.com/ouro-hoje#:~:text=O%20valor%20do%20grama%20do,em%20R%24%20282%2C25.')

gold_price =  navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
gold_price = float(gold_price.replace(',', '.'))
# print('gold: ' + str(gold_price) + ' ' + str(type(gold_price)))

navegador.quit() # close Chrome

In [30]:
# Passo 5

# importing table
import pandas as pd
table = pd.read_excel('Produtos.xlsx')

display(table)


,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [31]:
# Att price per unit
# nas linhas onde coluna 'moeda' == dolar
table.loc[table['Moeda'] == 'Dólar', 'Cotação'] = dolar_price
table.loc[table['Moeda'] == 'Euro', 'Cotação'] = euro_price
table.loc[table['Moeda'] == 'Ouro', 'Cotação'] = gold_price

display(table)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,4.966000,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,5.924016,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,4.966000,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,4.966000,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,5.924016,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,4.966000,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,283.950000,7000.00,1.15,8050.000


In [34]:
# Passo 6
# Att price in Reais
table['Preço Base Reais'] = table['Cotação']*table['Preço Base Original']

# Att final price
table['Preço Final'] = table['Margem']*table['Preço Base Reais']

display(table)


,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,4.966000,4965.950340,1.40,6952.330476
1,Carro Renault,4500.00,Euro,5.924016,26658.071505,2.00,53316.143010
2,Notebook Dell,899.99,Dólar,4.966000,4469.350340,1.70,7597.895578
3,IPhone,799.00,Dólar,4.966000,3967.834000,1.70,6745.317800
4,Carro Fiat,3000.00,Euro,5.924016,17772.047670,1.90,33766.890573
5,Celular Xiaomi,480.48,Dólar,4.966000,2386.063680,2.00,4772.127360
6,Joia 20g,20.00,Ouro,283.950000,5679.000000,1.15,6530.850000
